In [2]:
# !pip install yfinance # run this code to be able to access yfinance library.

In [5]:
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
import numpy as np

# Some libraries I thought we would need to use.
# Please feel free to add more that are needed